# A quick insight on Algebra and KZG Commitments

## Contents

1. `[Preliminaries](#preliminaries)
    - Number sets, prime numbers, coprime integers and modular aritmethic
    - Basic algebraic structures: Abelian groups, Rings, Fields, Galois Fields
        - finite groups, generators, the exponential map, pairings
		- Cryptographic groups: computational hardness assumptions
		- Polynomials
2. KZG Commitments
    - Setup
    - Commitment
    - Opening
    - Verification
    - A hand-made example
    - A `sagemath` implementation
3. Closing remarks

# 1. Preliminaries

( . . . )

# 2. KZG Commitments

( . . . )

## 2.6 A `sagemath` implementation

Let's replicate (and implement) the previous steps one by one.

> Remember, we want to prove that we know a Polynomial $\phi(x) = a_0 + a_1x + a_2x^2+\dots+a_tx^t$, where $\phi(x)\in\mathbb{F}_p[x]$.

### 2.6.1 Setup

First we'll have to define a couple variables:
- $p$ prime which will aid us to build our Prime Field $\mathbb{F}_p$
- $t$ the maximum degree of the polynomials in our Polynomial Field $\mathbb{F}_p[x]$, so that $t < p$

Remember that our prime needs $k$ bits of security! So we'll have to define a function to get a safe random prime number

In [1]:
def safe_random_prime(k):
    """Generate a safe random prime.
    Code taken from
    https://ask.sagemath.org/question/44765/randomly-generate-a-safe-prime-of-given-length/?answer=44766#post-id-44766
    """
    while true:
        p = random_prime(2^k - 1, false, 2^(k - 1))
        if ZZ((p - 1)/2).is_prime():
            return p

k = 16 # 16 bits of security. My laptop is not beefy and I want my public parameters to be computed in a matter of seconds
p = safe_random_prime(k)
print(f"Our safe random prime is {p}")

Our safe random prime is 44819


Let's generate a random $t$ so that $t < p$

In [2]:
from random import randint
t = randint(2, p) # 2 minimum to make it interesting
print(f"The maximum degree is {t}")

The maximum degree is 16351


Let's define the Prime Field $\mathbb{F}_p$

In [3]:
F_p = GF(p)
print(F_p)

Finite Field of size 44819


Now, let's get the generator $g$ of the Prime Field $\mathbb{F}_p$

In [4]:
g = F_p.multiplicative_generator()
print(f"The generator is {g}")

The generator is 2


Next, we have to define a pairing $e:\mathbb{G}\times\mathbb{G}\rightarrow\mathbb{G}_T$. We will define the simple pairing
$$
e(g_1,g_2)\mapsto g_1\cdot g_2 \quad\left(\text{mod}~p\right)
$$

In [5]:
def e(g1, g2):
    return F_p.prod(g1,g2)

We may also check that this pairing
- is bilinear:
$$
e(g_1^a,g_2^b)=e(g_1^{ab},g_2)=e(g_1,g_2^{ab})=e(g_1,g_2)^{ab}
$$
    see:
$$
ag_1\cdot bg_2 = abg_1\cdot g_2 = g_1\cdot abg_2 = ab(g_1\cdot g_2) \quad\left(\text{mod}~p\right)
$$ 
- non-degenerate
$$
e(g,g)\neq\boldsymbol{1}
$$
    see:
$$
2\cdot2 = 4 \neq 0 \quad\left(\text{mod}~p\right)
$$

At this point, the trusted party (us, haha) retrieves a random (and secret) $\alpha\in\mathbb{F}_p$

In [6]:
alpha = F_p.random_element()

This allows computing the Public Parameters `PP`

In [7]:
def compute_public_parameters(F_p, alpha, t):

    PP = []

    accumulated = 1
    for i in range(t + 1):
        PP.append(F_p.prod((
            accumulated, g
        )))
        accumulated = F_p.prod((accumulated, alpha))
    return PP

In [8]:
PP = compute_public_parameters(F_p, alpha, t)
PP

[2,
 21781,
 1423,
 12217,
 4037,
 19919,
 26319,
 31974,
 14036,
 26268,
 36796,
 159,
 6058,
 1081,
 7643,
 29618,
 37305,
 8277,
 32069,
 40206,
 26632,
 12047,
 35050,
 33421,
 18711,
 2562,
 24043,
 30103,
 8146,
 17212,
 14228,
 10751,
 38947,
 7697,
 35058,
 30907,
 24403,
 6611,
 40191,
 20141,
 23784,
 10651,
 25553,
 26185,
 6855,
 8333,
 14471,
 35231,
 10156,
 35445,
 10135,
 8430,
 17603,
 37018,
 42443,
 29554,
 12598,
 7560,
 44496,
 660,
 16690,
 21400,
 42719,
 32459,
 29696,
 35203,
 18955,
 15523,
 18373,
 41550,
 7651,
 27104,
 42997,
 12326,
 3398,
 30244,
 42270,
 5486,
 1556,
 4036,
 31438,
 3198,
 3456,
 34427,
 38718,
 1227,
 28991,
 43859,
 32766,
 34064,
 7129,
 34276,
 30146,
 5838,
 25397,
 30389,
 30318,
 41425,
 13318,
 5395,
 18948,
 6518,
 35802,
 21200,
 15931,
 24616,
 18109,
 34874,
 43910,
 27944,
 3122,
 27339,
 25172,
 22662,
 27097,
 33992,
 29755,
 27867,
 38524,
 39782,
 25417,
 24104,
 44548,
 29138,
 8869,
 25309,
 13224,
 12525,
 41705,
 149

Remember, once we have the public parameters, **we must delete `alpha`**.
_We are not going to do it though, for illustrative purposes!_

### 2.6.2 Commitment

It's time to do our commitment! In order to make it as applied as possible, let's commit a message. To do that though, we will some auxiliary code to help us!

#### Reed Solomon Codes

We're going to use Reed-Solomon encoding to transform our messages into a list of integers

In [9]:
from reedsolo import RSCodec

def reed_solomon_encode_string(input_string):

    # Initialize the Reed-Solomon encoder/decoder with a specific error correction level
    rs = RSCodec(10)  # You can adjust the error correction level as needed

    # Encode the input string
    encoded_bytes = rs.encode(input_string.encode())

    # Convert the encoded bytes to a list of integers
    return list(encoded_bytes)

We will also need to transform this into a list of tuples, which will contain coordinates

In [10]:
def message_as_coords(input_string):
    y = reed_solomon_encode_string(input_string)
    x = range(1, len(y) + 1)
    return list(zip(x, y))

Let's encode a message!

In [11]:
msg = message_as_coords("Hello Zepp!")
print(msg)

[(1, 72), (2, 101), (3, 108), (4, 108), (5, 111), (6, 32), (7, 90), (8, 101), (9, 112), (10, 112), (11, 33), (12, 25), (13, 255), (14, 128), (15, 142), (16, 165), (17, 163), (18, 57), (19, 193), (20, 169), (21, 195)]


#### Interpolating Polynomial
Cool! Let's create our Polynomial Ring $\mathbb{F}_p[x]$

In [12]:
F_p_X.<x> = PolynomialRing(F_p)
F_p_X

Univariate Polynomial Ring in x over Finite Field of size 44819

And get an interpolating polynomial over our coordinates (our message)

In [13]:
msg_poly = F_p_X.lagrange_polynomial(msg)
msg_poly

31364*x^20 + 3188*x^19 + 20603*x^18 + 39200*x^17 + 27756*x^16 + 42656*x^15 + 11232*x^14 + 39709*x^13 + 33453*x^12 + 13065*x^11 + 12501*x^10 + 25662*x^9 + 1068*x^8 + 2769*x^7 + 38250*x^6 + 1569*x^5 + 32069*x^4 + 44675*x^3 + 39666*x^2 + 16056*x + 16570

Since for $n$ points $(x,y)$ there exists one unique polynomial of degree $n-1$ that passes through all of them, we basically converted `"Hello Zep"` into a polynomial!

---

Okay, now we have the polynomial that we want to commit. We will need to evaluate the polynomial at $\alpha$. We will "_evaluate it on the exponent_". We can write a function for that

In [14]:
from functools import reduce

def exponent_evaluation(PP, poly, g):
    # Recover the Prime Field
    F_p = msg_poly.base_ring()
    # Get the Polynomial coefficients. They are given in ascending order
    poly_coefs = poly.coefficients()
    # Only keep the parameters needed
    pp_used = PP[:len(poly_coefs)]  
    
    return reduce(
        lambda a,b: F_p.sum(( # Sum each of the terms
            a,
            F_p.prod(( # Multiply...
                b[0],  # Coefficient
                b[1],  # corresponding \alpha^n
                g      # group generator
            ))
        )),
        zip(poly_coefs, pp_used),
        0
    )

In [15]:
commitment = exponent_evaluation(PP, msg_poly, g)
commitment

33231

This is our commitment!